# Comprehensive Analysis: AI Assistant Usage in Student Life

**Author**: Eman Toraih  
**Date**: January 5, 2026  
**Purpose**: Comprehensive statistical analysis and machine learning modeling of AI assistant usage patterns in student life

---

## Research Objectives
This comprehensive analysis explores patterns, predictors, and insights from student interactions with AI assistants. The analysis aims to:
- Understand usage patterns across student demographics and task types
- Identify factors that predict AI tool reuse
- Conduct statistical hypothesis testing
- Build predictive models for publication
- Generate actionable insights for educators and developers

## Dataset Overview
- **Source**: Kaggle - AI Assistant Usage in Student Life (Synthetic Dataset)
- **Sample Size**: 10,000 student-AI interaction sessions
- **Variables**: 11 features including demographics, session characteristics, and outcomes


## 1. Library Imports and Configuration


In [ ]:
# Core Data Manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistical Analysis
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway, mannwhitneyu
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                            roc_auc_score, roc_curve, confusion_matrix, classification_report,
                            precision_recall_curve, auc)
from sklearn.feature_selection import SelectKBest, f_classif
import xgboost as xgb
import lightgbm as lgb

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


## 2. Data Loading and Initial Screening


In [ ]:
# Load the dataset
df = pd.read_csv('ai_assistant_usage_student_life.csv')

# Convert SessionDate to datetime
df['SessionDate'] = pd.to_datetime(df['SessionDate'])

print("Dataset loaded successfully!")
print(f"\nDataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


In [ ]:
# Initial data inspection
print("=" * 60)
print("DATASET OVERVIEW")
print("=" * 60)
print("\nFirst 5 rows:")
display(df.head())

print("\n\nDataset Info:")
print(df.info())

print("\n\nBasic Statistics:")
display(df.describe())

print("\n\nData Types:")
print(df.dtypes)


In [ ]:
# Data Quality Check
print("=" * 60)
print("DATA QUALITY ASSESSMENT")
print("=" * 60)

print("\n1. Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing Percentage': missing_pct
})
display(missing_df[missing_df['Missing Count'] > 0])

print("\n2. Duplicate Rows:")
duplicates = df.duplicated().sum()
print(f"Total duplicates: {duplicates} ({duplicates/len(df)*100:.2f}%)")

print("\n3. Unique Values per Categorical Column:")
categorical_cols = df.select_dtypes(include=['object', 'bool']).columns
for col in categorical_cols:
    if col != 'SessionID':
        print(f"\n{col}:")
        print(df[col].value_counts())
        print(f"Unique values: {df[col].nunique()}")

print("\n4. Numerical Column Ranges:")
numerical_cols = df.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    print(f"\n{col}:")
    print(f"  Min: {df[col].min()}, Max: {df[col].max()}")
    print(f"  Mean: {df[col].mean():.2f}, Median: {df[col].median():.2f}")
    print(f"  Std: {df[col].std():.2f}")


## 3. Exploratory Data Analysis (EDA)

### 3.1 Target Variable Distribution

In [ ]:
# Target variable: UsedAgain
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
used_again_counts = df['UsedAgain'].value_counts()
axes[0].bar(used_again_counts.index.astype(str), used_again_counts.values, 
            color=['#ff6b6b', '#51cf66'], edgecolor='black', linewidth=1.5)
axes[0].set_xlabel('Used Again', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[0].set_title('Distribution of UsedAgain Variable', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Add count labels on bars
for i, v in enumerate(used_again_counts.values):
    axes[0].text(i, v + 50, str(v), ha='center', va='bottom', fontweight='bold')

# Pie chart
used_again_pct = df['UsedAgain'].value_counts(normalize=True) * 100
colors = ['#ff6b6b', '#51cf66']
axes[1].pie(used_again_pct.values, labels=[f'{label}\\n({pct:.1f}%)' 
                                           for label, pct in zip(used_again_pct.index, used_again_pct.values)],
            autopct='', colors=colors, startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('UsedAgain Distribution (Percentage)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Print statistics
print("UsedAgain Statistics:")
print(f"True: {used_again_counts[True]:,} ({used_again_pct[True]:.2f}%)")
print(f"False: {used_again_counts[False]:,} ({used_again_pct[False]:.2f}%)")
print(f"\nClass Imbalance Ratio: {used_again_counts[False]/used_again_counts[True]:.2f}")


### 3.2 Categorical Variables Analysis


In [ ]:
# Student Level distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Student Level
student_level_counts = df['StudentLevel'].value_counts()
axes[0, 0].barh(student_level_counts.index, student_level_counts.values, color='#339af0', edgecolor='black')
axes[0, 0].set_xlabel('Count', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Distribution by Student Level', fontsize=14, fontweight='bold')
axes[0, 0].grid(axis='x', alpha=0.3)
for i, v in enumerate(student_level_counts.values):
    axes[0, 0].text(v + 50, i, str(v), va='center', fontweight='bold')

# Task Type
task_type_counts = df['TaskType'].value_counts()
axes[0, 1].barh(task_type_counts.index, task_type_counts.values, color='#51cf66', edgecolor='black')
axes[0, 1].set_xlabel('Count', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Distribution by Task Type', fontsize=14, fontweight='bold')
axes[0, 1].grid(axis='x', alpha=0.3)
for i, v in enumerate(task_type_counts.values):
    axes[0, 1].text(v + 50, i, str(v), va='center', fontweight='bold')

# Discipline (top 10)
discipline_counts = df['Discipline'].value_counts().head(10)
axes[1, 0].barh(discipline_counts.index, discipline_counts.values, color='#ff922b', edgecolor='black')
axes[1, 0].set_xlabel('Count', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Top 10 Disciplines', fontsize=14, fontweight='bold')
axes[1, 0].grid(axis='x', alpha=0.3)
for i, v in enumerate(discipline_counts.values):
    axes[1, 0].text(v + 50, i, str(v), va='center', fontweight='bold')

# Final Outcome
outcome_counts = df['FinalOutcome'].value_counts()
axes[1, 1].barh(outcome_counts.index, outcome_counts.values, color='#ae3ec9', edgecolor='black')
axes[1, 1].set_xlabel('Count', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Distribution by Final Outcome', fontsize=14, fontweight='bold')
axes[1, 1].grid(axis='x', alpha=0.3)
for i, v in enumerate(outcome_counts.values):
    axes[1, 1].text(v + 50, i, str(v), va='center', fontweight='bold')

plt.tight_layout()
plt.show()


### 3.3 Numerical Variables Distribution


In [ ]:
# Distribution of numerical variables
numerical_cols = ['SessionLengthMin', 'TotalPrompts', 'AI_AssistanceLevel', 'SatisfactionRating']
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

for idx, col in enumerate(numerical_cols):
    row = idx // 2
    col_idx = idx % 2
    
    # Histogram with KDE
    axes[row, col_idx].hist(df[col], bins=50, color='#339af0', alpha=0.7, edgecolor='black')
    axes[row, col_idx].axvline(df[col].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df[col].mean():.2f}')
    axes[row, col_idx].axvline(df[col].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df[col].median():.2f}')
    axes[row, col_idx].set_xlabel(col, fontsize=12, fontweight='bold')
    axes[row, col_idx].set_ylabel('Frequency', fontsize=12, fontweight='bold')
    axes[row, col_idx].set_title(f'Distribution of {col}', fontsize=14, fontweight='bold')
    axes[row, col_idx].legend()
    axes[row, col_idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Box plots
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
for idx, col in enumerate(numerical_cols):
    bp = axes[idx].boxplot(df[col], patch_artist=True, 
                           boxprops=dict(facecolor='#339af0', alpha=0.7),
                           medianprops=dict(color='red', linewidth=2))
    axes[idx].set_ylabel(col, fontsize=12, fontweight='bold')
    axes[idx].set_title(f'Box Plot: {col}', fontsize=14, fontweight='bold')
    axes[idx].grid(axis='y', alpha=0.3)
    
plt.tight_layout()
plt.show()


### 3.4 Correlation Analysis


In [ ]:
# Correlation matrix for numerical variables
numerical_df = df[['SessionLengthMin', 'TotalPrompts', 'AI_AssistanceLevel', 
                   'SatisfactionRating']].copy()
numerical_df['UsedAgain'] = df['UsedAgain'].astype(int)

correlation_matrix = numerical_df.corr()

# Create heatmap
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, mask=mask,
            annot_kws={'fontsize': 11, 'fontweight': 'bold'})
plt.title('Correlation Matrix of Numerical Variables', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Print strong correlations
print("Strong Correlations (|r| > 0.5):")
print("=" * 50)
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.5:
            print(f"{correlation_matrix.columns[i]} vs {correlation_matrix.columns[j]}: {corr_val:.3f}")


### 3.5 Relationship Between Categorical Variables and UsedAgain


In [ ]:
# Analyze UsedAgain by categorical variables
categorical_vars = ['StudentLevel', 'TaskType', 'FinalOutcome']

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, var in enumerate(categorical_vars):
    # Calculate percentages
    crosstab = pd.crosstab(df[var], df['UsedAgain'], normalize='index') * 100
    
    # Create stacked bar chart
    crosstab.plot(kind='bar', stacked=True, ax=axes[idx], 
                  color=['#ff6b6b', '#51cf66'], edgecolor='black', linewidth=1)
    axes[idx].set_xlabel(var, fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Percentage (%)', fontsize=12, fontweight='bold')
    axes[idx].set_title(f'UsedAgain Distribution by {var}', fontsize=14, fontweight='bold')
    axes[idx].legend(['False', 'True'], title='Used Again', fontsize=10)
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Print detailed statistics
for var in categorical_vars:
    print(f"\n{'='*60}")
    print(f"UsedAgain by {var}")
    print('='*60)
    crosstab_counts = pd.crosstab(df[var], df['UsedAgain'], margins=True)
    crosstab_pct = pd.crosstab(df[var], df['UsedAgain'], normalize='index') * 100
    print("\nCounts:")
    display(crosstab_counts)
    print("\nPercentages:")
    display(crosstab_pct.round(2))


### 3.6 Relationship Between Numerical Variables and UsedAgain


In [ ]:
# Compare numerical variables by UsedAgain
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

for idx, col in enumerate(numerical_cols):
    row = idx // 2
    col_idx = idx % 2
    
    # Create grouped box plot
    data_to_plot = [df[df['UsedAgain']==False][col], df[df['UsedAgain']==True][col]]
    bp = axes[row, col_idx].boxplot(data_to_plot, labels=['False', 'True'],
                                    patch_artist=True,
                                    boxprops=dict(facecolor='lightblue', alpha=0.7),
                                    medianprops=dict(color='red', linewidth=2))
    axes[row, col_idx].set_xlabel('Used Again', fontsize=12, fontweight='bold')
    axes[row, col_idx].set_ylabel(col, fontsize=12, fontweight='bold')
    axes[row, col_idx].set_title(f'{col} by UsedAgain', fontsize=14, fontweight='bold')
    axes[row, col_idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical summary
print("\nStatistical Summary by UsedAgain:")
print("="*70)
for col in numerical_cols:
    print(f"\n{col}:")
    summary = df.groupby('UsedAgain')[col].agg(['mean', 'median', 'std', 'min', 'max'])
    display(summary)


## 4. Statistical Hypothesis Testing

### 4.1 Chi-Square Tests for Categorical Variables


In [ ]:
# Chi-square tests for independence
categorical_vars = ['StudentLevel', 'TaskType', 'FinalOutcome', 'Discipline']

chi2_results = []

for var in categorical_vars:
    contingency_table = pd.crosstab(df[var], df['UsedAgain'])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    # Cramer's V for effect size
    n = contingency_table.sum().sum()
    cramers_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))
    
    chi2_results.append({
        'Variable': var,
        'Chi-square': chi2,
        'p-value': p_value,
        'df': dof,
        "Cramer's V": cramers_v,
        'Significant': 'Yes' if p_value < 0.05 else 'No'
    })
    
    print(f"\n{'='*60}")
    print(f"Chi-square Test: {var} vs UsedAgain")
    print('='*60)
    print(f"Chi-square statistic: {chi2:.4f}")
    print(f"Degrees of freedom: {dof}")
    print(f"p-value: {p_value:.6f}")
    print(f"Cramer's V: {cramers_v:.4f}")
    print(f"Significant at α=0.05: {'Yes' if p_value < 0.05 else 'No'}")
    if p_value < 0.001:
        print("Interpretation: Very strong evidence of association")
    elif p_value < 0.01:
        print("Interpretation: Strong evidence of association")
    elif p_value < 0.05:
        print("Interpretation: Evidence of association")
    else:
        print("Interpretation: No significant association")

chi2_df = pd.DataFrame(chi2_results)
print("\n\nSummary of Chi-square Tests:")
print("="*70)
display(chi2_df)


### 4.2 T-tests and Non-parametric Tests for Numerical Variables


In [ ]:
# Compare numerical variables between UsedAgain groups
numerical_cols = ['SessionLengthMin', 'TotalPrompts', 'AI_AssistanceLevel', 'SatisfactionRating']

test_results = []

for col in numerical_cols:
    group_false = df[df['UsedAgain']==False][col].dropna()
    group_true = df[df['UsedAgain']==True][col].dropna()
    
    # Check normality (Shapiro-Wilk test on smaller sample for speed)
    sample_size = min(5000, len(group_false), len(group_true))
    _, p_norm_false = stats.shapiro(group_false.sample(sample_size)) if len(group_false) > 3 else (None, 0.01)
    _, p_norm_true = stats.shapiro(group_true.sample(sample_size)) if len(group_true) > 3 else (None, 0.01)
    is_normal = p_norm_false > 0.05 and p_norm_true > 0.05
    
    # Welch's t-test (doesn't assume equal variances)
    t_stat, p_t = stats.ttest_ind(group_true, group_false, equal_var=False)
    
    # Mann-Whitney U test (non-parametric)
    u_stat, p_u = mannwhitneyu(group_true, group_false, alternative='two-sided')
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt(((len(group_true)-1)*group_true.std()**2 + 
                          (len(group_false)-1)*group_false.std()**2) / 
                         (len(group_true) + len(group_false) - 2))
    cohens_d = (group_true.mean() - group_false.mean()) / pooled_std
    
    test_results.append({
        'Variable': col,
        'UsedAgain=True Mean': group_true.mean(),
        'UsedAgain=False Mean': group_false.mean(),
        'Mean Difference': group_true.mean() - group_false.mean(),
        'T-statistic': t_stat,
        'T-test p-value': p_t,
        'Mann-Whitney U': u_stat,
        'MW p-value': p_u,
        "Cohen's d": cohens_d,
        'Normal Distribution': 'Yes' if is_normal else 'No',
        'Significant (t-test)': 'Yes' if p_t < 0.05 else 'No'
    })
    
    print(f"\n{'='*70}")
    print(f"Statistical Tests: {col} by UsedAgain")
    print('='*70)
    print(f"UsedAgain=True:  Mean={group_true.mean():.3f}, SD={group_true.std():.3f}, N={len(group_true)}")
    print(f"UsedAgain=False: Mean={group_false.mean():.3f}, SD={group_false.std():.3f}, N={len(group_false)}")
    print(f"\nWelch's t-test: t={t_stat:.4f}, p={p_t:.6f}")
    print(f"Mann-Whitney U: U={u_stat:.4f}, p={p_u:.6f}")
    print(f"Cohen's d: {cohens_d:.4f}")
    
    if abs(cohens_d) < 0.2:
        effect_size = "negligible"
    elif abs(cohens_d) < 0.5:
        effect_size = "small"
    elif abs(cohens_d) < 0.8:
        effect_size = "medium"
    else:
        effect_size = "large"
    print(f"Effect size: {effect_size}")
    
    if p_t < 0.05:
        print(f"Interpretation: Significant difference (p < 0.05)")
    else:
        print(f"Interpretation: No significant difference (p ≥ 0.05)")

test_df = pd.DataFrame(test_results)
print("\n\nSummary of Statistical Tests:")
print("="*70)
display(test_df.round(4))


### 4.3 ANOVA for Multiple Groups


In [ ]:
# ANOVA for numerical variables across categorical groups
anova_results = []

# Test SatisfactionRating across TaskType
groups_task = [df[df['TaskType']==task]['SatisfactionRating'].dropna() 
               for task in df['TaskType'].unique()]
f_stat_task, p_anova_task = f_oneway(*groups_task)
eta_squared_task = f_stat_task / (f_stat_task + (len(df) - len(df['TaskType'].unique())))

anova_results.append({
    'Variable': 'SatisfactionRating',
    'Group Variable': 'TaskType',
    'F-statistic': f_stat_task,
    'p-value': p_anova_task,
    'Eta-squared': eta_squared_task,
    'Significant': 'Yes' if p_anova_task < 0.05 else 'No'
})

print("ANOVA: SatisfactionRating across TaskType")
print("="*60)
print(f"F-statistic: {f_stat_task:.4f}")
print(f"p-value: {p_anova_task:.6f}")
print(f"Eta-squared: {eta_squared_task:.4f}")
print(f"Significant at α=0.05: {'Yes' if p_anova_task < 0.05 else 'No'}")

# Test SessionLengthMin across StudentLevel
groups_level = [df[df['StudentLevel']==level]['SessionLengthMin'].dropna() 
                for level in df['StudentLevel'].unique()]
f_stat_level, p_anova_level = f_oneway(*groups_level)
eta_squared_level = f_stat_level / (f_stat_level + (len(df) - len(df['StudentLevel'].unique())))

anova_results.append({
    'Variable': 'SessionLengthMin',
    'Group Variable': 'StudentLevel',
    'F-statistic': f_stat_level,
    'p-value': p_anova_level,
    'Eta-squared': eta_squared_level,
    'Significant': 'Yes' if p_anova_level < 0.05 else 'No'
})

print("\n\nANOVA: SessionLengthMin across StudentLevel")
print("="*60)
print(f"F-statistic: {f_stat_level:.4f}")
print(f"p-value: {p_anova_level:.6f}")
print(f"Eta-squared: {eta_squared_level:.4f}")
print(f"Significant at α=0.05: {'Yes' if p_anova_level < 0.05 else 'No'}")

anova_df = pd.DataFrame(anova_results)
print("\n\nSummary of ANOVA Tests:")
print("="*70)
display(anova_df.round(4))


## 5. Time Series Analysis

### 5.1 Temporal Trends


In [ ]:
# Extract temporal features
df['Year'] = df['SessionDate'].dt.year
df['Month'] = df['SessionDate'].dt.month
df['DayOfWeek'] = df['SessionDate'].dt.day_name()
df['WeekOfYear'] = df['SessionDate'].dt.isocalendar().week

# Time series analysis
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Sessions over time (monthly)
monthly_counts = df.groupby([df['SessionDate'].dt.to_period('M')]).size()
axes[0, 0].plot(monthly_counts.index.astype(str), monthly_counts.values, 
                marker='o', linewidth=2, markersize=8, color='#339af0')
axes[0, 0].set_xlabel('Month', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Number of Sessions', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Total Sessions Over Time (Monthly)', fontsize=14, fontweight='bold')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(alpha=0.3)

# UsedAgain rate over time
monthly_used = df.groupby([df['SessionDate'].dt.to_period('M'), 'UsedAgain']).size().unstack(fill_value=0)
monthly_used_pct = (monthly_used[True] / (monthly_used[True] + monthly_used[False]) * 100)
axes[0, 1].plot(monthly_used_pct.index.astype(str), monthly_used_pct.values,
                marker='o', linewidth=2, markersize=8, color='#51cf66')
axes[0, 1].set_xlabel('Month', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('UsedAgain Rate (%)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('UsedAgain Rate Over Time', fontsize=14, fontweight='bold')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(alpha=0.3)
axes[0, 1].axhline(y=monthly_used_pct.mean(), color='red', linestyle='--', 
                   label=f'Mean: {monthly_used_pct.mean():.1f}%')
axes[0, 1].legend()

# Sessions by day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_counts = df['DayOfWeek'].value_counts().reindex(day_order, fill_value=0)
axes[1, 0].bar(range(len(day_counts)), day_counts.values, color='#ff922b', edgecolor='black')
axes[1, 0].set_xticks(range(len(day_counts)))
axes[1, 0].set_xticklabels(day_counts.index, rotation=45, ha='right')
axes[1, 0].set_ylabel('Number of Sessions', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Sessions by Day of Week', fontsize=14, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)

# Average SatisfactionRating over time
monthly_satisfaction = df.groupby(df['SessionDate'].dt.to_period('M'))['SatisfactionRating'].mean()
axes[1, 1].plot(monthly_satisfaction.index.astype(str), monthly_satisfaction.values,
                marker='o', linewidth=2, markersize=8, color='#ae3ec9')
axes[1, 1].set_xlabel('Month', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Average Satisfaction Rating', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Average Satisfaction Rating Over Time', fontsize=14, fontweight='bold')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(alpha=0.3)
axes[1, 1].axhline(y=monthly_satisfaction.mean(), color='red', linestyle='--',
                   label=f'Mean: {monthly_satisfaction.mean():.2f}')
axes[1, 1].legend()

plt.tight_layout()
plt.show()


## 7. Machine Learning Models

### 7.1 Data Preparation for ML


In [ ]:
# Create a copy for feature engineering
df_fe = df.copy()

# Feature engineering
print("Creating new features...")

# 1. Efficiency metrics
df_fe['MinPerPrompt'] = df_fe['SessionLengthMin'] / df_fe['TotalPrompts']
df_fe['MinPerPrompt'] = df_fe['MinPerPrompt'].replace([np.inf, -np.inf], np.nan)

# 2. Satisfaction per prompt
df_fe['SatisfactionPerPrompt'] = df_fe['SatisfactionRating'] / df_fe['TotalPrompts']
df_fe['SatisfactionPerPrompt'] = df_fe['SatisfactionPerPrompt'].replace([np.inf, -np.inf], np.nan)

# 3. Session intensity (prompts per minute)
df_fe['PromptsPerMin'] = df_fe['TotalPrompts'] / df_fe['SessionLengthMin']
df_fe['PromptsPerMin'] = df_fe['PromptsPerMin'].replace([np.inf, -np.inf], np.nan)

# 4. Binary features
df_fe['HighAssistance'] = (df_fe['AI_AssistanceLevel'] >= 3).astype(int)
df_fe['HighSatisfaction'] = (df_fe['SatisfactionRating'] >= 4).astype(int)
df_fe['LongSession'] = (df_fe['SessionLengthMin'] >= df_fe['SessionLengthMin'].median()).astype(int)

# 5. Outcome success
df_fe['SuccessfulOutcome'] = (df_fe['FinalOutcome'] == 'Assignment Completed').astype(int)

# 6. Temporal features (already created)
# Year, Month, DayOfWeek, WeekOfYear

print(f"Original features: {len(df.columns)}")
print(f"Total features after engineering: {len(df_fe.columns)}")
print(f"\nNew features created:")
new_features = [col for col in df_fe.columns if col not in df.columns]
for feat in new_features:
    print(f"  - {feat}")

# Display summary of new numerical features
print("\n\nSummary of new numerical features:")
new_num_features = [f for f in new_features if df_fe[f].dtype in [np.float64, np.int64]]
display(df_fe[new_num_features].describe())


## 7. Machine Learning Models

### 7.1 Data Preparation for ML


In [ ]:
# Prepare data for machine learning
# Select features for modeling
feature_cols = ['StudentLevel', 'Discipline', 'TaskType', 'FinalOutcome',
                'SessionLengthMin', 'TotalPrompts', 'AI_AssistanceLevel', 
                'SatisfactionRating', 'MinPerPrompt', 'SatisfactionPerPrompt',
                'HighAssistance', 'HighSatisfaction', 'LongSession', 'SuccessfulOutcome']

# Create feature dataframe
X = df_fe[feature_cols].copy()
y = df_fe['UsedAgain'].astype(int)

# Handle categorical variables with one-hot encoding
categorical_cols = ['StudentLevel', 'Discipline', 'TaskType', 'FinalOutcome']
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

print(f"Original feature count: {len(feature_cols)}")
print(f"Feature count after encoding: {X_encoded.shape[1]}")
print(f"Target variable distribution:")
print(y.value_counts())
print(f"\nClass imbalance ratio: {y.value_counts()[0] / y.value_counts()[1]:.3f}")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Scale numerical features
numerical_cols_ml = [col for col in X_encoded.columns 
                     if X_encoded[col].dtype in [np.float64, np.int64] 
                     and col not in ['HighAssistance', 'HighSatisfaction', 'LongSession', 'SuccessfulOutcome']]

scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[numerical_cols_ml] = scaler.fit_transform(X_train[numerical_cols_ml])
X_test_scaled[numerical_cols_ml] = scaler.transform(X_test[numerical_cols_ml])

print(f"\nScaling applied to {len(numerical_cols_ml)} numerical features")


### 7.2 Baseline Models

In [ ]:
# Train and evaluate multiple baseline models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=42, eval_metric='logloss'),
    'LightGBM': lgb.LGBMClassifier(random_state=42, verbose=-1)
}

results = []

for name, model in models.items():
    print(f"\n{'='*60}")
    print(f"Training {name}...")
    print('='*60)
    
    # Use scaled data for Logistic Regression, original for tree-based
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)
    
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc
    })
    
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"ROC-AUC:   {roc_auc:.4f}")

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('ROC-AUC', ascending=False)

print("\n\n" + "="*70)
print("MODEL COMPARISON SUMMARY")
print("="*70)
display(results_df.round(4))


### 7.3 Model Visualization


In [ ]:
# Visualize model performance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar plot of metrics
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
x = np.arange(len(results_df))
width = 0.15

for i, metric in enumerate(metrics):
    axes[0].bar(x + i*width, results_df[metric], width, label=metric, alpha=0.8)

axes[0].set_xlabel('Model', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Score', fontsize=12, fontweight='bold')
axes[0].set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
axes[0].set_xticks(x + width * 2)
axes[0].set_xticklabels(results_df['Model'], rotation=45, ha='right')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)
axes[0].set_ylim([0, 1])

# ROC-AUC comparison
axes[1].barh(results_df['Model'], results_df['ROC-AUC'], color='#339af0', edgecolor='black')
axes[1].set_xlabel('ROC-AUC Score', fontsize=12, fontweight='bold')
axes[1].set_title('ROC-AUC by Model', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)
axes[1].set_xlim([0, 1])

for i, v in enumerate(results_df['ROC-AUC']):
    axes[1].text(v + 0.01, i, f'{v:.4f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()


### 7.4 ROC Curves for Best Models


In [ ]:
# Plot ROC curves for top 3 models
top_models = results_df.head(3)['Model'].tolist()

fig, ax = plt.subplots(figsize=(10, 8))

for name in top_models:
    model = models[name]
    
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]
    
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = roc_auc_score(y_test, y_proba)
    
    ax.plot(fpr, tpr, linewidth=2, label=f'{name} (AUC = {roc_auc:.4f})')

ax.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
ax.set_xlabel('False Positive Rate', fontsize=12, fontweight='bold')
ax.set_ylabel('True Positive Rate', fontsize=12, fontweight='bold')
ax.set_title('ROC Curves - Top 3 Models', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=11)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()


### 7.7 Cross-Validation Results


In [ ]:
# Cross-validation for model robustness assessment
print("="*70)
print("CROSS-VALIDATION ANALYSIS")
print("="*70)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_results = []

# Evaluate top 3 models with cross-validation
top_3_models = results_df.head(3)['Model'].tolist()

for name in top_3_models:
    print(f"\n{name} - Cross-Validation:")
    print("-"*60)
    
    if name == 'Logistic Regression':
        model = LogisticRegression(max_iter=1000, random_state=42)
        X_cv = X_train_scaled
    elif name == 'Random Forest':
        model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        X_cv = X_train
    elif name == 'XGBoost':
        model = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
        X_cv = X_train
    elif name == 'LightGBM':
        model = lgb.LGBMClassifier(random_state=42, verbose=-1)
        X_cv = X_train
    elif name == 'Gradient Boosting':
        model = GradientBoostingClassifier(random_state=42)
        X_cv = X_train
    else:
        continue
    
    # Cross-validation scores
    cv_scores_auc = cross_val_score(model, X_cv, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
    cv_scores_acc = cross_val_score(model, X_cv, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
    cv_scores_f1 = cross_val_score(model, X_cv, y_train, cv=cv, scoring='f1', n_jobs=-1)
    
    cv_results.append({
        'Model': name,
        'CV_ROC-AUC_Mean': cv_scores_auc.mean(),
        'CV_ROC-AUC_Std': cv_scores_auc.std(),
        'CV_Accuracy_Mean': cv_scores_acc.mean(),
        'CV_Accuracy_Std': cv_scores_acc.std(),
        'CV_F1_Mean': cv_scores_f1.mean(),
        'CV_F1_Std': cv_scores_f1.std()
    })
    
    print(f"ROC-AUC: {cv_scores_auc.mean():.4f} (+/- {cv_scores_auc.std() * 2:.4f})")
    print(f"Accuracy: {cv_scores_acc.mean():.4f} (+/- {cv_scores_acc.std() * 2:.4f})")
    print(f"F1-Score: {cv_scores_f1.mean():.4f} (+/- {cv_scores_f1.std() * 2:.4f})")
    print(f"CV Scores (ROC-AUC): {cv_scores_auc}")

cv_results_df = pd.DataFrame(cv_results)
print("\n\n" + "="*70)
print("CROSS-VALIDATION SUMMARY")
print("="*70)
display(cv_results_df.round(4))

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics_cv = ['CV_ROC-AUC_Mean', 'CV_Accuracy_Mean', 'CV_F1_Mean']
metric_labels = ['ROC-AUC', 'Accuracy', 'F1-Score']

for idx, (metric, label) in enumerate(zip(metrics_cv, metric_labels)):
    means = cv_results_df[metric]
    stds = cv_results_df[metric.replace('_Mean', '_Std')]
    
    axes[idx].barh(cv_results_df['Model'], means, xerr=stds, 
                   capsize=5, alpha=0.7, edgecolor='black')
    axes[idx].set_xlabel(f'{label} Score', fontsize=12, fontweight='bold')
    axes[idx].set_title(f'Cross-Validation {label}', fontsize=14, fontweight='bold')
    axes[idx].grid(axis='x', alpha=0.3)
    
    for i, (m, s) in enumerate(zip(means, stds)):
        axes[idx].text(m + s + 0.01, i, f'{m:.3f}±{s:.3f}', 
                      va='center', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()


### 7.8 Model Calibration Analysis


In [ ]:
# Model Calibration Analysis
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import brier_score_loss

print("="*70)
print("MODEL CALIBRATION ANALYSIS")
print("="*70)

# Select best model for calibration analysis
best_model_name_cv = cv_results_df.loc[cv_results_df['CV_ROC-AUC_Mean'].idxmax(), 'Model']
print(f"\nAnalyzing calibration for: {best_model_name_cv}")

# Train and get predictions
if best_model_name_cv == 'Logistic Regression':
    best_model_cal = LogisticRegression(max_iter=1000, random_state=42)
    best_model_cal.fit(X_train_scaled, y_train)
    y_proba_cal = best_model_cal.predict_proba(X_test_scaled)[:, 1]
elif best_model_name_cv == 'Random Forest':
    best_model_cal = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    best_model_cal.fit(X_train, y_train)
    y_proba_cal = best_model_cal.predict_proba(X_test)[:, 1]
elif best_model_name_cv == 'XGBoost':
    best_model_cal = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
    best_model_cal.fit(X_train, y_train)
    y_proba_cal = best_model_cal.predict_proba(X_test)[:, 1]
else:
    best_model_cal = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    best_model_cal.fit(X_train, y_train)
    y_proba_cal = best_model_cal.predict_proba(X_test)[:, 1]

# Calibration curve
fraction_of_positives, mean_predicted_value = calibration_curve(
    y_test, y_proba_cal, n_bins=10, strategy='uniform'
)

# Brier score
brier_score = brier_score_loss(y_test, y_proba_cal)

# Plot calibration curve
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Calibration curve
axes[0].plot(mean_predicted_value, fraction_of_positives, "s-", 
             label=f'{best_model_name_cv}', linewidth=2, markersize=8)
axes[0].plot([0, 1], [0, 1], "k:", label="Perfectly calibrated", linewidth=2)
axes[0].set_xlabel('Mean Predicted Probability', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Fraction of Positives', fontsize=12, fontweight='bold')
axes[0].set_title(f'Calibration Curve - {best_model_name_cv}', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3)

# Histogram of predicted probabilities
axes[1].hist(y_proba_cal, bins=20, edgecolor='black', alpha=0.7, color='#339af0')
axes[1].axvline(y_proba_cal.mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {y_proba_cal.mean():.3f}')
axes[1].set_xlabel('Predicted Probability', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[1].set_title(f'Predicted Probability Distribution - {best_model_name_cv}', 
                  fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nBrier Score: {brier_score:.4f}")
print("(Lower is better; 0 = perfect calibration, 1 = worst)")
print(f"\nInterpretation:")
if brier_score < 0.1:
    print("Excellent calibration - model predictions are well-calibrated")
elif brier_score < 0.2:
    print("Good calibration - model predictions are reasonably reliable")
else:
    print("Poor calibration - model predictions may not be reliable")

# Calibrated model (using Platt scaling)
print(f"\n{'='*70}")
print("CALIBRATED MODEL (Platt Scaling)")
print("="*70)

if best_model_name_cv == 'Logistic Regression':
    calibrated_model = CalibratedClassifierCV(best_model_cal, method='sigmoid', cv=3)
    calibrated_model.fit(X_train_scaled, y_train)
    y_proba_calibrated = calibrated_model.predict_proba(X_test_scaled)[:, 1]
else:
    calibrated_model = CalibratedClassifierCV(best_model_cal, method='isotonic', cv=3)
    calibrated_model.fit(X_train, y_train)
    y_proba_calibrated = calibrated_model.predict_proba(X_test)[:, 1]

brier_score_calibrated = brier_score_loss(y_test, y_proba_calibrated)
fraction_of_positives_cal, mean_predicted_value_cal = calibration_curve(
    y_test, y_proba_calibrated, n_bins=10, strategy='uniform'
)

print(f"Brier Score (Calibrated): {brier_score_calibrated:.4f}")
print(f"Improvement: {brier_score - brier_score_calibrated:.4f}")

# Plot comparison
plt.figure(figsize=(10, 8))
plt.plot(mean_predicted_value, fraction_of_positives, "s-", 
         label=f'{best_model_name_cv} (Original)', linewidth=2, markersize=8)
plt.plot(mean_predicted_value_cal, fraction_of_positives_cal, "o-", 
         label=f'{best_model_name_cv} (Calibrated)', linewidth=2, markersize=8)
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated", linewidth=2)
plt.xlabel('Mean Predicted Probability', fontsize=12, fontweight='bold')
plt.ylabel('Fraction of Positives', fontsize=12, fontweight='bold')
plt.title('Calibration Comparison: Original vs Calibrated', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


### 7.9 SHAP Values for Model Interpretability


In [ ]:
# SHAP Values for Model Interpretability
try:
    import shap
    shap_available = True
except ImportError:
    shap_available = False
    print("SHAP library not available. Install with: pip install shap")

if shap_available:
    print("="*70)
    print("SHAP VALUE ANALYSIS")
    print("="*70)
    
    # Use best tree-based model for SHAP (works best with tree models)
    if best_model_name_cv in ['Random Forest', 'XGBoost', 'LightGBM', 'Gradient Boosting']:
        if best_model_name_cv == 'Random Forest':
            shap_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
            shap_model.fit(X_train, y_train)
            X_shap = X_train
            explainer = shap.TreeExplainer(shap_model)
        elif best_model_name_cv == 'XGBoost':
            shap_model = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
            shap_model.fit(X_train, y_train)
            X_shap = X_train
            explainer = shap.TreeExplainer(shap_model)
        elif best_model_name_cv == 'LightGBM':
            shap_model = lgb.LGBMClassifier(random_state=42, verbose=-1)
            shap_model.fit(X_train, y_train)
            X_shap = X_train
            explainer = shap.TreeExplainer(shap_model)
        else:
            shap_model = GradientBoostingClassifier(random_state=42)
            shap_model.fit(X_train, y_train)
            X_shap = X_train
            explainer = shap.TreeExplainer(shap_model)
        
        # Calculate SHAP values (use sample for efficiency)
        sample_size = min(500, len(X_test))
        X_shap_sample = X_shap.sample(n=sample_size, random_state=42)
        shap_values = explainer.shap_values(X_shap_sample)
        
        # For binary classification, use class 1 (UsedAgain=True)
        if isinstance(shap_values, list):
            shap_values_display = shap_values[1]
        else:
            shap_values_display = shap_values
        
        print(f"\nSHAP values calculated for {best_model_name_cv}")
        print(f"Sample size: {sample_size} observations")
        print(f"Number of features: {X_shap_sample.shape[1]}")
        
        # Summary plot
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values_display, X_shap_sample, 
                         show=False, max_display=20, plot_size=(12, 8))
        plt.title(f'SHAP Summary Plot - {best_model_name_cv}', 
                 fontsize=14, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.show()
        
        # Bar plot of mean SHAP values
        shap.summary_plot(shap_values_display, X_shap_sample, plot_type="bar", 
                         show=False, max_display=20)
        plt.title(f'Mean |SHAP Value| - {best_model_name_cv}', 
                 fontsize=14, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.show()
        
        # Calculate feature importance from SHAP
        shap_importance = pd.DataFrame({
            'Feature': X_shap_sample.columns,
            'Mean_|SHAP_Value|': np.abs(shap_values_display).mean(axis=0)
        }).sort_values('Mean_|SHAP_Value|', ascending=False).head(15)
        
        print("\nTop 15 Features by Mean |SHAP Value|:")
        display(shap_importance.round(4))
        
    else:
        print(f"\nSHAP analysis optimized for tree-based models.")
        print(f"Using Random Forest as proxy for SHAP analysis...")
        shap_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        shap_model.fit(X_train, y_train)
        explainer = shap.TreeExplainer(shap_model)
        
        sample_size = min(500, len(X_test))
        X_shap_sample = X_train.sample(n=sample_size, random_state=42)
        shap_values = explainer.shap_values(X_shap_sample)
        
        if isinstance(shap_values, list):
            shap_values_display = shap_values[1]
        else:
            shap_values_display = shap_values
        
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values_display, X_shap_sample, 
                         show=False, max_display=20, plot_size=(12, 8))
        plt.title('SHAP Summary Plot - Random Forest (Proxy)', 
                 fontsize=14, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.show()
        
        shap_importance = pd.DataFrame({
            'Feature': X_shap_sample.columns,
            'Mean_|SHAP_Value|': np.abs(shap_values_display).mean(axis=0)
        }).sort_values('Mean_|SHAP_Value|', ascending=False).head(15)
        
        print("\nTop 15 Features by Mean |SHAP Value|:")
        display(shap_importance.round(4))

else:
    print("\n" + "="*70)
    print("SHAP ANALYSIS - LIBRARY NOT INSTALLED")
    print("="*70)
    print("\nTo enable SHAP analysis, install the library:")
    print("  pip install shap")
    print("\nSHAP provides model interpretability by explaining individual predictions.")


### 7.5 Feature Importance (Random Forest)

In [ ]:
# Train Random Forest and get feature importance
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Get feature importance
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False).head(20)

# Plot feature importance
plt.figure(figsize=(12, 8))
plt.barh(range(len(feature_importance)), feature_importance['Importance'], 
         color='#339af0', edgecolor='black')
plt.yticks(range(len(feature_importance)), feature_importance['Feature'])
plt.xlabel('Feature Importance', fontsize=12, fontweight='bold')
plt.title('Top 20 Most Important Features (Random Forest)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

for i, v in enumerate(feature_importance['Importance']):
    plt.text(v + 0.001, i, f'{v:.4f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nTop 10 Most Important Features:")
display(feature_importance.head(10))


### 7.6 Hyperparameter Tuning (Best Model)


In [ ]:
# Hyperparameter tuning for the best model (assuming XGBoost or Random Forest)
best_model_name = results_df.iloc[0]['Model']
print(f"Performing hyperparameter tuning for: {best_model_name}")

if best_model_name == 'XGBoost':
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.8, 1.0]
    }
    base_model = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
elif best_model_name == 'Random Forest':
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }
    base_model = RandomForestClassifier(random_state=42, n_jobs=-1)
else:
    print(f"Hyperparameter tuning not implemented for {best_model_name}")
    param_grid = None

if param_grid:
    # Use smaller grid search for demonstration (full search takes longer)
    print("Performing grid search (this may take a few minutes)...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = GridSearchCV(
        base_model, param_grid, cv=cv, 
        scoring='roc_auc', n_jobs=-1, verbose=1
    )
    grid_search.fit(X_train, y_train)
    
    print(f"\nBest parameters: {grid_search.best_params_}")
    print(f"Best CV score: {grid_search.best_score_:.4f}")
    
    # Evaluate on test set
    best_model = grid_search.best_estimator_
    y_pred_tuned = best_model.predict(X_test)
    y_proba_tuned = best_model.predict_proba(X_test)[:, 1]
    
    accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
    roc_auc_tuned = roc_auc_score(y_test, y_proba_tuned)
    f1_tuned = f1_score(y_test, y_pred_tuned)
    
    print(f"\nTuned Model Performance on Test Set:")
    print(f"Accuracy: {accuracy_tuned:.4f}")
    print(f"ROC-AUC:  {roc_auc_tuned:.4f}")
    print(f"F1-Score: {f1_tuned:.4f}")
    
    print(f"\nImprovement over baseline:")
    print(f"ROC-AUC: {roc_auc_tuned - results_df[results_df['Model']==best_model_name]['ROC-AUC'].values[0]:.4f}")


## 8. Key Insights and Conclusions

### 8.1 Summary of Key Findings


In [ ]:
# Generate comprehensive insights
print("="*70)
print("COMPREHENSIVE ANALYSIS SUMMARY")
print("="*70)

print("\n1. DATASET OVERVIEW")
print("-"*70)
print(f"   • Total sessions analyzed: {len(df):,}")
print(f"   • Time period: {df['SessionDate'].min().strftime('%Y-%m-%d')} to {df['SessionDate'].max().strftime('%Y-%m-%d')}")
print(f"   • Student levels: {', '.join(df['StudentLevel'].unique())}")
print(f"   • Task types: {', '.join(df['TaskType'].unique())}")

print("\n2. TARGET VARIABLE (UsedAgain)")
print("-"*70)
used_again_pct = df['UsedAgain'].value_counts(normalize=True) * 100
print(f"   • Reuse rate: {used_again_pct[True]:.1f}% ({df['UsedAgain'].sum():,} sessions)")
print(f"   • Non-reuse rate: {used_again_pct[False]:.1f}% ({(~df['UsedAgain']).sum():,} sessions)")
print(f"   • Class imbalance: Moderate ({used_again_pct[False]/used_again_pct[True]:.2f}:1)")

print("\n3. KEY DEMOGRAPHIC PATTERNS")
print("-"*70)
# Most common student level
most_common_level = df['StudentLevel'].value_counts().index[0]
print(f"   • Most common student level: {most_common_level}")
# Most common task type
most_common_task = df['TaskType'].value_counts().index[0]
print(f"   • Most common task type: {most_common_task}")
# Average session length
print(f"   • Average session length: {df['SessionLengthMin'].mean():.1f} minutes")
print(f"   • Average prompts per session: {df['TotalPrompts'].mean():.1f}")

print("\n4. STATISTICAL SIGNIFICANCE")
print("-"*70)
# Check which tests were significant
print("   Significant associations with UsedAgain:")
sig_vars = []
if len(chi2_df) > 0:
    for _, row in chi2_df.iterrows():
        if row['Significant'] == 'Yes':
            sig_vars.append(f"{row['Variable']} (χ²)")
if len(test_df) > 0:
    for _, row in test_df.iterrows():
        if row['Significant (t-test)'] == 'Yes':
            sig_vars.append(f"{row['Variable']} (t-test)")

if sig_vars:
    for var in sig_vars[:5]:  # Show first 5
        print(f"     - {var}")
else:
    print("     (Run statistical tests section for details)")

print("\n5. MACHINE LEARNING PERFORMANCE")
print("-"*70)
if len(results_df) > 0:
    best_model = results_df.iloc[0]
    print(f"   • Best performing model: {best_model['Model']}")
    print(f"   • Accuracy: {best_model['Accuracy']:.1%}")
    print(f"   • ROC-AUC: {best_model['ROC-AUC']:.3f}")
    print(f"   • F1-Score: {best_model['F1-Score']:.3f}")
else:
    print("     (Run ML models section for details)")

print("\n6. KEY CORRELATIONS")
print("-"*70)
corr_with_used = correlation_matrix['UsedAgain'].drop('UsedAgain').abs().sort_values(ascending=False)
if len(corr_with_used) > 0:
    print(f"   • Strongest predictor: {corr_with_used.index[0]} (r={corr_with_used.iloc[0]:.3f})")

print("\n7. PRACTICAL IMPLICATIONS")
print("-"*70)
print("   • High reuse rate suggests AI assistants are generally well-received")
print("   • Session characteristics and outcomes are key predictors")
print("   • Feature engineering improved model interpretability")
print("   • Models can help identify at-risk user segments")

print("\n" + "="*70)


---**Analysis completed!** This notebook provides a comprehensive foundation for manuscript preparation and further research.---## Citation and Attribution**Notebook Author**: Eman Toraih  **Date Created**: January 5, 2026  **Analysis Type**: Comprehensive statistical analysis with machine learning modeling**Key Features**:- Rigorous statistical hypothesis testing with effect sizes- Temporal trend analysis- Cross-validated machine learning models- Model calibration assessment- SHAP values for interpretability- Publication-ready methodology**Contact**: emantoraih@outlook.com; toraihe@upstate.eduFor questions or collaborations, please refer to the GitHub repository or contact the author.